In [ ]:
import string
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
def remove_non_verb(text):
    pos_tagged = nltk.pos_tag(text)
    verbs = [pos_tagged[0][0]] + [i[0] for i in pos_tagged[1:] if 'VB' in i[1]] # always include 1st word as nltk doesnt perform well here
    return verbs

In [ ]:
text = nltk.word_tokenize("'divide evenly between two")
remove_non_verb(text)

In [ ]:
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize('brown', pos='v'))
wn.synsets('brown')